# Import Libraries

In [ ]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,Settings
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.llms.ollama import Ollama

# Read Document

In [ ]:
documents = SimpleDirectoryReader('data').load_data()


# Sepicfy Embedding and LLM model

In [ ]:
Settings.embed_model = OllamaEmbedding(model_name='nomic-embed-text')
Settings.llm = Ollama(model='phi3',request_timeout=360.0)

# Create Nodes

``` Nodes are chunks of data along with their metadata which also include the relationship between the chunks. It specify how your data is to be splitted into smalller components for indexing pupose```

In [ ]:
from llama_index.core.node_parser import SentenceSplitter,SemanticSplitterNodeParser

splitter = SentenceSplitter(chunk_size=1024)
nodes = splitter.get_nodes_from_documents(documents)

# Build Index

### Here we build two different indexex, one for summarization task and other for question answering task. This is where the concept of AGENTIC-RAG begins```

In [ ]:
from llama_index.core import SummaryIndex, VectorStoreIndex

In [ ]:
vector_index = VectorStoreIndex(nodes)   ## for question answering task
summary_index = SummaryIndex(nodes)      ## for summarization task

# Build Query Engines

### Two different engines are created for two different tasks. These engines will be used by be decision making agent (later steps) for performing specialized task

In [ ]:
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize"
)

vector_query_engine = vector_index.as_query_engine()

# Buld Query Engine Tools

``` A way to anotate query engine or create customized engines```

The descriptions are the annotation used by agent to decide which engine to select for a given task

In [ ]:
from llama_index.core.tools import QueryEngineTool

summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=("Useful for summarization questions related to the document"),

)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=("Useful for retrieving specific context form the document"),
    
)

# Build Router

```The Router or the agent responsible for routing user query to task specific engine. It uses different kind of selector to forward a user query to one or more releavnt engines```

In [ ]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

In [ ]:
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool
    ],
    verbose=True
)

In [ ]:
response = query_engine.query("What is the summary of the document?")

In [ ]:
print(str(response))

In [ ]:
response = query_engine.query("What is multi head attention?")

In [ ]:
print(str(response))